### Team s_toppo's solution

This Notebook scores around LB 1400 for two weeks(2020/12/02).  
Note: This is not our final/best model, which is too big to train in Kaggle Notebook. But the overall idea remains the same.

Special thanks to SaltyFish.

You can see this agent's episode here.  
https://www.kaggle.com/c/google-football/submissions?dialog=episodes-submission-18036600

In [ ]:
import numpy as np
from numpy import linalg as LA

import matplotlib.pyplot as plt
import pandas as pd
import json
import glob
import seaborn as sns

from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import GroupKFold

from tqdm import tqdm

import math

## load json

In [ ]:
TEAMNAME = "SaltyFish"
replay_dir = "../input/salty-375/"

In [ ]:
action_set_v1=[
"idle","left","top_left","top","top_right","right","bottom_right","bottom","bottom_left","long_pass","high_pass","short_pass","shot","sprint","release_direction","release_sprint","sliding","dribble","release_dribble"
]

In [ ]:
json_paths=[]
json_info_paths=[]

for path in glob.glob(replay_dir+"*"):
    if path.count("info")!=0:
        json_info_paths.append(path)
    else:
        json_paths.append(path)

print("replay num: {}".format(len(json_paths)))

## create df

In [ ]:
def calc_dist(x1,y1,x2,y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

def get_angle_rad(x0,y0,x1,y1,x2,y2): # radian
    vec1=np.array([x1-x0,y1-y0])
    vec2=np.array([x2-x0,y2-y0])
    
    i = np.inner(vec1, vec2)
    n = LA.norm(vec1) * LA.norm(vec2)
    c = i / n
    a = np.arccos(np.clip(c, -1.0, 1.0))
    return a

def get_angle(x0,y0,x1,y1,x2,y2):
    vec1=np.array([x1-x0,y1-y0])
    vec2=np.array([x2-x0,y2-y0])
    
    i = np.inner(vec1, vec2)
    n = LA.norm(vec1) * LA.norm(vec2)
    c = i / n
    a = np.rad2deg(np.arccos(np.clip(c, -1.0, 1.0)))
    
    return a

def get_angle_to_goal(x,y):
    #return get_angle(x,y,1,0.044,1,-0.044)
    return get_angle(x,y,1,0.044*1.5,1,-0.044*1.5)

def get_dist_opp_goal(x,y):
    return calc_dist(x,y,1,0)

def pressure_to_xy(x,y,team):
    pressure_x=0
    pressure_y=0
    abs_pressure=0
    for i in range(len(team)):
        opp_x = team[i][0]
        opp_y = team[i][1]

        dist = calc_dist(x,y,opp_x,opp_y)
        pressure_x+=(x - opp_x)/dist
        pressure_y+=(y - opp_y)/dist
        
        abs_pressure += calc_dist(pressure_x,pressure_y,0,0)
        
    return pressure_x,pressure_y,abs_pressure

def clamp(x,mn,mx):
    return min(mx, max(x, mn))

def normalizedClamp(x,mn,mx):
    ret = clamp(x,mn,mx)
    return (ret - mn) / (mx - mn);

def normalize(x,y): # x**2 + y**2 -> 1
    dist = (x**2 + y**2)**0.5
    if dist==0:
        return x,y
    return x/dist, y/dist 

def get_pass_score(px,py, pdir_x, pdir_y, target_x, target_y, target_dir_x,target_dir_y):
    INPUT_POWER=0.4 # change later
    autoDirectionBias = 0.3 # shortpass
    
    norm_x, norm_y = normalize(pdir_x,pdir_y)
    
    po = INPUT_POWER*60/55
    
    manualTarget_x = px + norm_x * po
    manualTarget_y = py + norm_y * po
    
    passDuration = 0.3 + calc_dist(px,py,target_x,target_y)*55 * 0.05
    passDuration = ((clamp(passDuration,0,1) ** 0.7)*0.7)*10
    
    targetPos_x = target_x + target_dir_x*passDuration
    targetPos_y = target_y + target_dir_y*passDuration
    
    target_manual_dist = calc_dist(manualTarget_x,manualTarget_y,targetPos_x,targetPos_y) * 55
    
    distanceRating = (normalizedClamp(target_manual_dist,0,70)**0.8)*0.8
    
    angleRating = abs(get_angle_rad(px,py,manualTarget_x,manualTarget_y,targetPos_x,targetPos_y))/3.14159
    
    
    final_x = targetPos_x * autoDirectionBias + manualTarget_x * (1-autoDirectionBias)
    final_y = targetPos_y * autoDirectionBias + manualTarget_y * (1-autoDirectionBias)
    
    return distanceRating+angleRating,final_x,final_y

def get_lowest_pass_score(player_ind, team, team_dir):
    min_ind=-1
    min_score=2
    
    player_x = team[player_ind][0]
    player_y = team[player_ind][1]
    
    player_dir1 = team_dir[player_ind][0]
    player_dir2 = team_dir[player_ind][1]
    
    final_x = -1
    final_y = -1
    
    for ind in range(len(team)):
        if ind == player_ind:
            continue

        target_x = team[ind][0]
        target_y = team[ind][1]

        target_dir1 = team_dir[ind][0]
        target_dir2 = team_dir[ind][1]

        score,tx,ty = get_pass_score(player_x,player_y,player_dir1,player_dir2,target_x,target_y,target_dir1,target_dir2)
        if score < min_score:
            min_ind = ind
            min_score = score
            final_x = tx
            final_y = ty
    return min_ind,min_score,final_x,final_y

def my_atan2(x,y): # return atan(y/x)
    if x==0:
        return 0
    return math.atan2(y,x)

def convert_raw_y(raw): # y*=1.5
    raw["ball"][1] *= 1.5
    raw["ball_direction"][1] *= 1.5
    
    for ind in range(len(raw["left_team"])):
        raw["left_team"][ind][1] *= 1.5
        raw["left_team_direction"][ind][1] *= 1.5
    for ind in range(len(raw["right_team"])):
        raw["right_team"][ind][1] *= 1.5
        raw["right_team_direction"][ind][1] *= 1.5
    return raw

def invert_raw_y(raw): # y=-y
    raw["ball"][1] = -raw["ball"][1]
    raw["ball_direction"][1] = -raw["ball_direction"][1] 
    
    for ind in range(len(raw["left_team"])):
        raw["left_team"][ind][1]            =-raw["left_team"][ind][1]
        raw["left_team_direction"][ind][1]  =-raw["left_team_direction"][ind][1]
    for ind in range(len(raw["right_team"])):
        raw["right_team"][ind][1]           =-raw["right_team"][ind][1]
        raw["right_team_direction"][ind][1] =-raw["right_team_direction"][ind][1]
    return raw


def invert_act_y(action):
    if action==2:
        return 8
    if action==8:
        return 2
    if action==7:
        return 3 
    if action==3:
        return 7
    if action==6:
        return 4
    if action==4:
        return 6
    return action

In [ ]:
def create_dict(raw):
   
    d=dict()
    
    ball = raw["ball"]
    d["ballx"] = ball[0]
    d["bally"] = ball[1]
    d["ballz"] = ball[2]
    
    d["ballx_d"] = raw["ball_direction"][0]
    d["bally_d"] = raw["ball_direction"][1]
    d["ballz_d"] = raw["ball_direction"][2]
    
    d["ball_velo"]=(raw["ball_direction"][0]**2 + 
                    raw["ball_direction"][1]**2 + 
                    raw["ball_direction"][2]**2)**0.5
    
    d["ballr_1"]=raw["ball_rotation"][0]
    d["ballr_2"]=raw["ball_rotation"][1]
    d["ballr_3"]=raw["ball_rotation"][2]
    
    d["ball_owned_team"]=raw["ball_owned_team"]
    
    d["ball_10step_x"]=ball[0] + 10*raw["ball_direction"][0]
    d["ball_10step_y"]=ball[1] + 10*raw["ball_direction"][1]
    
    d["ball_20step_x"]=ball[0] + 20*raw["ball_direction"][0]
    d["ball_20step_y"]=ball[1] + 20*raw["ball_direction"][1]
    
    d["ball_angle_goal"]=get_angle_to_goal(ball[0],ball[1])
    d["ball_10step_angle_goal"]=get_angle_to_goal(ball[0] + 10*raw["ball_direction"][0],ball[1] + 10*raw["ball_direction"][1])
    
    d["game_mode"]=raw["game_mode"]
    
    player_ind = raw["active"]
    
    if raw["ball_owned_team"]==0 and raw["active"]==raw["ball_owned_player"]:
        d["own_and_active"]=1
    else:
        d["own_and_active"]=0

    player_x = raw["left_team"][player_ind][0]
    player_y = raw["left_team"][player_ind][1]
    
    d["player_x"]=player_x
    d["player_y"]=player_y
    
    d["active_dir1"]=raw["left_team_direction"][player_ind][0]
    d["active_dir2"]=raw["left_team_direction"][player_ind][1]    
    
    d["active_tired"]=raw["left_team_tired_factor"][player_ind]
    d["active_role"]=raw["left_team_roles"][player_ind]
    d["active_yellow"] = raw["left_team_yellow_card"][player_ind]
    
    d["ball_dist_xy"]=calc_dist(ball[0],ball[1],player_x,player_y)
    d["ball_10step_x_diff"]=player_x-(ball[0] + 10*raw["ball_direction"][0])
    d["ball_10step_y_diff"]=player_y-(ball[1] + 10*raw["ball_direction"][1])

    d["ball_20step_x_diff"]=player_x-(ball[0] + 20*raw["ball_direction"][0])
    d["ball_20step_y_diff"]=player_y-(ball[1] + 20*raw["ball_direction"][1])
    
    d["ball_dir1"]=player_x-ball[0]
    d["ball_dir2"]=player_y-ball[1]

    
    d["dist_ally_goal"]=((player_x+1)**2 + player_y**2)**0.5
    d["dist_opp_goal"]=((player_x-1)**2 + player_y**2)**0.5
    
    ball_dist_goal = get_dist_opp_goal(ball[0],ball[1])
    
    ally_pos_x=[]
    ally_near_opp_goal_cnt=0
    min_ind=-1
    min_dist=10**10
    for ind in range(len(raw["left_team"])):
        now_x = raw["left_team"][ind][0]
        now_y = raw["left_team"][ind][1]
        ally_pos_x.append(now_x)
        if get_dist_opp_goal(now_x,now_y) < ball_dist_goal:
            ally_near_opp_goal_cnt+=1

        if ind == player_ind:
            continue
        dist = (player_x-now_x)**2 + (player_y-now_y)**2
        if dist < min_dist:
            min_ind=ind
            min_dist=dist    
    
    ally_pos_x.sort()
    d["ally_offside_line"]=ally_pos_x[1]
    d["ally_upper_mean_x"] = np.mean(ally_pos_x[:len(ally_pos_x)//2])
    d["ally_lower_mean_x"] = np.mean(ally_pos_x[len(ally_pos_x)//2:])
    
    d["num_ally_near_opp_goal"]=ally_near_opp_goal_cnt
    d["nearest_ally_dist"]=min_dist**0.5
    d["nearest_ally_x"]=raw["left_team"][min_ind][0]
    d["nearest_ally_y"]=raw["left_team"][min_ind][1]
    d["nearest_ally_dir1"]=raw["left_team_direction"][min_ind][0]
    d["nearest_ally_dir2"]=raw["left_team_direction"][min_ind][1]
    
    opp_near_opp_goal_cnt=0
    min_ind=-1
    min_dist=10**10
    opp_pos_x = []
    for ind in range(len(raw["right_team"])):
        now_x = raw["right_team"][ind][0]
        now_y = raw["right_team"][ind][1]
        if get_dist_opp_goal(now_x,now_y) < ball_dist_goal:
            opp_near_opp_goal_cnt+=1

        opp_pos_x.append(now_x)

        dist = (player_x-now_x)**2 + (player_y-now_y)**2
        if dist < min_dist:
            min_ind=ind
            min_dist=dist
    
    d["num_opp_near_opp_goal"]=opp_near_opp_goal_cnt
    d["nearest_opp_dist_player"]=min_dist**0.5 
    d["nearest_opp_x"]=raw["right_team"][min_ind][0]
    d["nearest_opp_y"]=raw["right_team"][min_ind][1]
    d["nearest_opp_dir1"]=raw["right_team_direction"][min_ind][0]
    d["nearest_opp_dir2"]=raw["right_team_direction"][min_ind][1]
    
    min_ind=-1
    min_dist=10**10
    for ind in range(len(raw["right_team"])):
        now_x = raw["right_team"][ind][0]
        now_y = raw["right_team"][ind][1]

        dist = calc_dist(ball[0],ball[1],now_x,now_y)

        if dist < min_dist:
            min_ind=ind
            min_dist=dist  

    min_x=raw["right_team"][min_ind][0]
    min_y=raw["right_team"][min_ind][1]
    
    d["ball_nearest_opp_dist_ball"]=min_dist
    d["ball_nearest_opp_dist_player"]=calc_dist(min_x,min_y,player_x,player_y)
    d["ball_nearest_opp_x"]=min_x
    d["ball_nearest_opp_y"]=min_y
    d["ball_nearest_opp_dir1"]=raw["left_team_direction"][min_ind][0]
    d["ball_nearest_opp_dir2"]=raw["left_team_direction"][min_ind][1]

    gk_ind = raw["right_team_roles"][0]
    gk_x = raw["right_team"][gk_ind][0]
    gk_y = raw["right_team"][gk_ind][1]
    
    d["opp_keeper_x"]=gk_x
    d["opp_keeper_y"]=gk_y
    d["opp_keeper_x_player"]=player_x - gk_x
    d["opp_keeper_y_player"]=player_y - gk_y
    d["opp_keeper_dir1"]=raw["right_team_direction"][gk_ind][0]
    d["opp_keeper_dir2"]=raw["right_team_direction"][gk_ind][1]
    
    opp_pos_x.sort()
    d["offside_line"]=opp_pos_x[-2]
    d["opp_upper_mean_x"] = np.mean(opp_pos_x[:len(opp_pos_x)//2])
    d["opp_lower_mean_x"] = np.mean(opp_pos_x[len(opp_pos_x)//2:])
    d["steps_left"]=raw["steps_left"]
    d["score"]=raw["score"][0]-raw["score"][1]
        
    d["player_pressure_from_opp_x"],d["player_pressure_from_opp_y"],d["player_pressure_abs"] = pressure_to_xy(player_x,player_y,raw["right_team"])
    d["ball_pressure_from_opp_x"],d["ball_pressure_from_opp_y"],d["ball_pressure_from_opp_abs"] = pressure_to_xy(ball[0],ball[1],raw["right_team"])
    d["ball_pressure_from_ally_x"],d["ball_pressure_from_ally_y"],d["ball_pressure_from_ally_abs"] = pressure_to_xy(ball[0],ball[1],raw["left_team"])
    
    d["player_angle_dir_opp_goal"]=get_angle(player_x,player_y,1,0,d["active_dir1"],d["active_dir2"])
    d["player_angle_dir_ally_goal"]=get_angle(player_x,player_y,-1,0,d["active_dir1"],d["active_dir2"])
    
    min_ind, score,pass_x,pass_y= get_lowest_pass_score(raw["active"], raw["left_team"], raw["left_team_direction"])
    
    d["pass_x"] = pass_x
    d["pass_y"] = pass_y
    
    d["pass_score"] = score
    
    p1,p2,d["pass_pressure_from_opp_abs"]  = pressure_to_xy(pass_x,pass_y,raw["left_team"])    
    p1,p2,d["pass_pressure_from_ally_abs"] = pressure_to_xy(pass_x,pass_y,raw["right_team"])
    
    d["pass_dist_from_player"] = calc_dist(pass_x,pass_y,player_x,player_y)
    d["pass_dist_from_ball"] = calc_dist(pass_x,pass_y,ball[0],ball[1])
    
    d["ballx_d_norm"], d["bally_d_norm"] = normalize(d["ballx_d"],d["bally_d"])
    d["active_dir1_norm"], d["active_dir2_norm"] = normalize(d["active_dir1"], d["active_dir2"])
    d["ball_10step_x_diff_norm"], d["ball_10step_y_diff_norm"] = normalize(d["ball_10step_x_diff"], d["ball_10step_y_diff"])
    d["ball_20step_x_diff_norm"], d["ball_20step_y_diff_norm"] = normalize(d["ball_20step_x_diff"], d["ball_20step_y_diff"])
    d["ball_dir1_norm"], d["ball_dir2_norm"] = normalize(d["ball_dir1"], d["ball_dir2"])
    d["nearest_ally_dir1_norm"], d["nearest_ally_dir2_norm"] = normalize(d["nearest_ally_dir1"], d["nearest_ally_dir2"])
    d["nearest_opp_dir1_norm"], d["nearest_opp_dir2_norm"] = normalize(d["nearest_opp_dir1"], d["nearest_opp_dir2"])
    d["ball_nearest_opp_dir1_norm"], d["ball_nearest_opp_dir2_norm"] = normalize(d["ball_nearest_opp_dir1"], d["ball_nearest_opp_dir2"])
    
    d["opp_keeper_x_player_norm"], d["opp_keeper_y_player_norm"] = normalize(d["opp_keeper_x_player"], d["opp_keeper_y_player"])
    d["opp_keeper_dir1_norm"], d["opp_keeper_dir2_norm"] = normalize(d["opp_keeper_dir1"], d["opp_keeper_dir2"])
    
    d["ball_d_theta"] = my_atan2(d["ballx_d_norm"], d["bally_d_norm"])
    d["active_dir_theta"] = my_atan2(d["active_dir1_norm"], d["active_dir2_norm"])
    d["ball_10step_theta"] = my_atan2(d["ball_10step_x_diff_norm"], d["ball_10step_y_diff_norm"])
    d["ball_20step_theta"] = my_atan2(d["ball_20step_x_diff_norm"], d["ball_20step_y_diff_norm"])
    d["ball_dir_theta"] = my_atan2( d["ball_dir1_norm"], d["ball_dir2_norm"])
    d["nearest_ally_dir_theta"] = my_atan2(d["nearest_ally_dir1_norm"], d["nearest_ally_dir2_norm"])
    d["nearest_opp_dir_theta"] = my_atan2(d["nearest_opp_dir1_norm"], d["nearest_opp_dir2_norm"])
    d["ball_nearest_opp_dir_theta"] = my_atan2(d["ball_nearest_opp_dir1_norm"], d["ball_nearest_opp_dir2_norm"])
    d["opp_keeper_theta"] = my_atan2(d["opp_keeper_x_player_norm"], d["opp_keeper_y_player_norm"])
    d["opp_keeper_dir_theta"] = my_atan2( d["opp_keeper_dir1_norm"], d["opp_keeper_dir2_norm"])
    
    d["sticky_sprint"] = raw["sticky_actions"][8]
    sub_feats=['ball_dist_xy___nearest_opp_dist_player',       'dist_ally_goal___ball_nearest_opp_dist_ball',       'dist_ally_goal___ball_nearest_opp_dist_player',       'ball_nearest_opp_dist_ball___ball_nearest_opp_dist_player',       'ballx___player_x', 'ballx___nearest_opp_x',       'ballx___opp_keeper_x', 'ballx_d___active_dir1',       'ballx_d___ball_dir1', 'ball_10step_x___player_x',       'ball_10step_x___opp_keeper_x_player',       'ball_20step_x___offside_line',       'ball_20step_x___opp_upper_mean_x',       'player_x___ally_lower_mean_x', 'player_x___nearest_opp_x',       'player_x___offside_line', 'active_dir1___ball_dir1',       'active_dir1___nearest_opp_dir1',       'active_dir1___opp_keeper_dir1',       'ball_10step_x_diff___opp_keeper_dir1',       'ball_20step_x_diff___opp_keeper_x',       'ball_dir1___nearest_opp_dir1',       'ball_dir1___ball_nearest_opp_dir1',       'ball_dir1___opp_keeper_dir1',       'ally_offside_line___ally_lower_mean_x',       'ally_upper_mean_x___offside_line',       'ally_upper_mean_x___opp_upper_mean_x',       'ally_lower_mean_x___offside_line',       'ally_lower_mean_x___opp_upper_mean_x',       'opp_keeper_x_player___opp_keeper_dir1',       'offside_line___opp_upper_mean_x', 'bally___player_y',       'bally___nearest_opp_y', 'bally_d___active_dir2',       'bally_d___ball_dir2', 'player_y___nearest_opp_y',       'active_dir2___ball_10step_y_diff',       'active_dir2___opp_keeper_dir2',       'ball_10step_y_diff___opp_keeper_dir2',       'ball_dir2___nearest_opp_dir2',       'ball_dir2___opp_keeper_dir2',       'opp_keeper_y___opp_keeper_y_player']
    
    for l in sub_feats:
        l1,l2 = l.split("___")
        d[l] = d[l1] - d[l2]
    
    return d    

In [ ]:
%%time 

act_lis =[]
replay_group = []
rows_list = [] # list of dict

replays = []
for replay_ind, path_replay in enumerate(tqdm(json_paths)):
    json_open = open(path_replay, 'r')
    json_load = json.load(json_open)
    
    for frame in range(len(json_load["steps"])-1):
        
        if TEAMNAME in json_load["info"]["TeamNames"][0]:
            team=0
        elif TEAMNAME in json_load["info"]["TeamNames"][1]:
            team=1
        else:
            raise BaseException("teamname{} not found!".format(TEAMNAME))
            
        raw = json_load["steps"][frame][team]["observation"]["players_raw"][0]
        raw = convert_raw_y(raw)
        
        action = json_load["steps"][frame+1][team]["action"][0]
        
        # ignore idle, release_sprint, release_direction, release_dribble, dribble
        if action == 0 or action == 14 or action == 15 or action == 17 or action == 18 or action == 16:
            continue
        if action == 5 and frame%3==0:
            continue
            
        d = create_dict(raw)
        rows_list.append(d)
        
        replay_group.append(replay_ind)
               
        act_lis.append(action)

In [ ]:
%%time
X_df = pd.DataFrame(rows_list)

In [ ]:
print(X_df.shape)
X_df.head()

In [ ]:
import collections
c = collections.Counter(act_lis)
for act,cnt in c.most_common():
    print(cnt,action_set_v1[act])

## CatBoost Train

In [ ]:
group_kfold = GroupKFold(n_splits=5)
for train_index, test_index in group_kfold.split(X_df, act_lis, replay_group):
    print(len(test_index))

In [ ]:
test_index

In [ ]:
x_train = X_df.iloc[train_index, :]
x_test  = X_df.iloc[test_index, :]

y_train = np.array(act_lis)[train_index]
y_test  = np.array(act_lis)[test_index]

In [ ]:
cat_features = ["game_mode","active_role","ball_owned_team","own_and_active","sticky_sprint"]
train_dataset = Pool(data=x_train,
                     label=y_train,
                     cat_features=cat_features)

eval_dataset = Pool(data=x_test,
                    label=y_test,
                    cat_features=cat_features)

In [ ]:
params = {
      'depth': 8,
      'learning_rate': 0.1,
      'random_seed': 42,
   #   'early_stopping_rounds':100, 
      'iterations': 2500,
      'task_type': 'GPU',
      "use_best_model":True,
}

In [ ]:
%%time
model = CatBoostClassifier(**params)
model.fit(train_dataset, eval_set=eval_dataset)

In [ ]:
y_pred = model.predict(x_test,prediction_type="Probability")
y_pred_max = np.argmax(y_pred, axis=1)

In [ ]:
y_pred_max+=1 # because no idle

In [ ]:
accuracy = sum(y_test == y_pred_max) / len(y_test)
print("Accuracy : {}".format(accuracy))

from sklearn.metrics import fbeta_score
print("F1 : {}".format(fbeta_score(y_test, y_pred_max, average='macro', beta=0.5)))

In [ ]:
action_set_v1=[
"idle","left","top_left","top","top_right","right","bottom_right","bottom","bottom_left","long_pass","high_pass","short_pass","shot","sprint","release_direction","release_sprint","sliding","dribble","release_dribble"
]

action_set_v1_used=[]
for i in np.unique(y_pred_max):
    action_set_v1_used.append(action_set_v1[i])
len(action_set_v1_used)

In [ ]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns=300

In [ ]:
feature_importance = model.get_feature_importance()

importance_df = pd.DataFrame({
    "feature":x_test.columns,
    "importance":feature_importance
})
importance_df.sort_values("importance",ascending=False)

In [ ]:
from sklearn import metrics
res = metrics.classification_report(y_test, y_pred_max, digits=3,output_dict=True,target_names = action_set_v1_used)
df = pd.DataFrame(res).transpose()
df=df.drop(['accuracy', 'macro avg', 'weighted avg'])
df.sort_values("f1-score")

## Save

In [ ]:
fname = TEAMNAME + "_saved.model"
model.save_model(fname)

In [ ]:
!ls -lh

## Submit

In [ ]:
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.7 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib
!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.7.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
!mkdir -p /kaggle_simulations/agent
!cp $fname /kaggle_simulations/agent/model.model
!ls /kaggle_simulations/agent

In [ ]:
%%writefile /kaggle_simulations/agent/main.py

import numpy as np
from numpy import linalg as LA
import os
import math

from catboost import CatBoostClassifier, Pool
import pandas as pd

model = CatBoostClassifier()
model.load_model('/kaggle_simulations/agent/model.model')

def calc_dist(x1,y1,x2,y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

def get_angle_rad(x0,y0,x1,y1,x2,y2): # radian
    vec1=np.array([x1-x0,y1-y0])
    vec2=np.array([x2-x0,y2-y0])
    
    i = np.inner(vec1, vec2)
    n = LA.norm(vec1) * LA.norm(vec2)
    c = i / n
    a = np.arccos(np.clip(c, -1.0, 1.0))
    return a

def get_angle(x0,y0,x1,y1,x2,y2):
    vec1=np.array([x1-x0,y1-y0])
    vec2=np.array([x2-x0,y2-y0])
    
    i = np.inner(vec1, vec2)
    n = LA.norm(vec1) * LA.norm(vec2)
    c = i / n
    a = np.rad2deg(np.arccos(np.clip(c, -1.0, 1.0)))
    
    return a

def get_angle_to_goal(x,y):
    #return get_angle(x,y,1,0.044,1,-0.044)
    return get_angle(x,y,1,0.044*1.5,1,-0.044*1.5)

def get_dist_opp_goal(x,y):
    return calc_dist(x,y,1,0)

def pressure_to_xy(x,y,team):
    pressure_x=0
    pressure_y=0
    abs_pressure=0
    for i in range(len(team)):
        opp_x = team[i][0]
        opp_y = team[i][1]

        dist = calc_dist(x,y,opp_x,opp_y)
        pressure_x+=(x - opp_x)/dist
        pressure_y+=(y - opp_y)/dist
        
        abs_pressure += calc_dist(pressure_x,pressure_y,0,0)
        
    return pressure_x,pressure_y,abs_pressure

def clamp(x,mn,mx):
    return min(mx, max(x, mn))

def normalizedClamp(x,mn,mx):
    ret = clamp(x,mn,mx)
    return (ret - mn) / (mx - mn);

def normalize(x,y): # x**2 + y**2 -> 1
    dist = (x**2 + y**2)**0.5
    if dist==0:
        return x,y
    return x/dist, y/dist 

def get_pass_score(px,py, pdir_x, pdir_y, target_x, target_y, target_dir_x,target_dir_y):
    INPUT_POWER=0.4 # change later
    autoDirectionBias = 0.3 # shortpass
    
    norm_x, norm_y = normalize(pdir_x,pdir_y)
    
    po = INPUT_POWER*60/55
    
    manualTarget_x = px + norm_x * po
    manualTarget_y = py + norm_y * po
    
    passDuration = 0.3 + calc_dist(px,py,target_x,target_y)*55 * 0.05
    passDuration = ((clamp(passDuration,0,1) ** 0.7)*0.7)*10
    
    targetPos_x = target_x + target_dir_x*passDuration
    targetPos_y = target_y + target_dir_y*passDuration
    
    target_manual_dist = calc_dist(manualTarget_x,manualTarget_y,targetPos_x,targetPos_y) * 55
    
    distanceRating = (normalizedClamp(target_manual_dist,0,70)**0.8)*0.8
    
    angleRating = abs(get_angle_rad(px,py,manualTarget_x,manualTarget_y,targetPos_x,targetPos_y))/3.14159
    
    
    final_x = targetPos_x * autoDirectionBias + manualTarget_x * (1-autoDirectionBias)
    final_y = targetPos_y * autoDirectionBias + manualTarget_y * (1-autoDirectionBias)
    
    return distanceRating+angleRating,final_x,final_y

def get_lowest_pass_score(player_ind, team, team_dir):
    min_ind=-1
    min_score=2
    
    player_x = team[player_ind][0]
    player_y = team[player_ind][1]
    
    player_dir1 = team_dir[player_ind][0]
    player_dir2 = team_dir[player_ind][1]
    
    final_x = -1
    final_y = -1
    
    for ind in range(len(team)):
        if ind == player_ind:
            continue

        target_x = team[ind][0]
        target_y = team[ind][1]

        target_dir1 = team_dir[ind][0]
        target_dir2 = team_dir[ind][1]

        score,tx,ty = get_pass_score(player_x,player_y,player_dir1,player_dir2,target_x,target_y,target_dir1,target_dir2)
        if score < min_score:
            min_ind = ind
            min_score = score
            final_x = tx
            final_y = ty
    return min_ind,min_score,final_x,final_y

def my_atan2(x,y): # return atan(y/x)
    if x==0:
        return 0
    return math.atan2(y,x)

def convert_raw_y(raw): # y*=1.5
    raw["ball"][1] *= 1.5
    raw["ball_direction"][1] *= 1.5
    
    for ind in range(len(raw["left_team"])):
        raw["left_team"][ind][1] *= 1.5
        raw["left_team_direction"][ind][1] *= 1.5
    for ind in range(len(raw["right_team"])):
        raw["right_team"][ind][1] *= 1.5
        raw["right_team_direction"][ind][1] *= 1.5
    return raw

def invert_raw_y(raw): # y=-y
    raw["ball"][1] = -raw["ball"][1]
    raw["ball_direction"][1] = -raw["ball_direction"][1] 
    
    for ind in range(len(raw["left_team"])):
        raw["left_team"][ind][1]            =-raw["left_team"][ind][1]
        raw["left_team_direction"][ind][1]  =-raw["left_team_direction"][ind][1]
    for ind in range(len(raw["right_team"])):
        raw["right_team"][ind][1]           =-raw["right_team"][ind][1]
        raw["right_team_direction"][ind][1] =-raw["right_team_direction"][ind][1]
    return raw

def invert_act_y(action):
    if action==2:
        return 8
    if action==8:
        return 2
    if action==7:
        return 3 
    if action==3:
        return 7
    if action==6:
        return 4
    if action==4:
        return 6
    return action

def create_df(raw):   
    
    d=dict()
    
    ball = raw["ball"]
    d["ballx"] = ball[0]
    d["bally"] = ball[1]
    d["ballz"] = ball[2]
    
    d["ballx_d"] = raw["ball_direction"][0]
    d["bally_d"] = raw["ball_direction"][1]
    d["ballz_d"] = raw["ball_direction"][2]
    
    d["ball_velo"]=(raw["ball_direction"][0]**2 + 
                    raw["ball_direction"][1]**2 + 
                    raw["ball_direction"][2]**2)**0.5
    
    d["ballr_1"]=raw["ball_rotation"][0]
    d["ballr_2"]=raw["ball_rotation"][1]
    d["ballr_3"]=raw["ball_rotation"][2]
    
    d["ball_owned_team"]=raw["ball_owned_team"]
    
    d["ball_10step_x"]=ball[0] + 10*raw["ball_direction"][0]
    d["ball_10step_y"]=ball[1] + 10*raw["ball_direction"][1]
    
    d["ball_20step_x"]=ball[0] + 20*raw["ball_direction"][0]
    d["ball_20step_y"]=ball[1] + 20*raw["ball_direction"][1]
    
    d["ball_angle_goal"]=get_angle_to_goal(ball[0],ball[1])
    d["ball_10step_angle_goal"]=get_angle_to_goal(ball[0] + 10*raw["ball_direction"][0],ball[1] + 10*raw["ball_direction"][1])
    
    d["game_mode"]=raw["game_mode"]
    
    player_ind = raw["active"]
    
    if raw["ball_owned_team"]==0 and raw["active"]==raw["ball_owned_player"]:
        d["own_and_active"]=1
    else:
        d["own_and_active"]=0

    player_x = raw["left_team"][player_ind][0]
    player_y = raw["left_team"][player_ind][1]
    
    d["player_x"]=player_x
    d["player_y"]=player_y
    
    d["active_dir1"]=raw["left_team_direction"][player_ind][0]
    d["active_dir2"]=raw["left_team_direction"][player_ind][1]    
    
    d["active_tired"]=raw["left_team_tired_factor"][player_ind]
    d["active_role"]=raw["left_team_roles"][player_ind]
    d["active_yellow"] = raw["left_team_yellow_card"][player_ind]
    
    d["ball_dist_xy"]=calc_dist(ball[0],ball[1],player_x,player_y)
    d["ball_10step_x_diff"]=player_x-(ball[0] + 10*raw["ball_direction"][0])
    d["ball_10step_y_diff"]=player_y-(ball[1] + 10*raw["ball_direction"][1])
    d["ball_20step_x_diff"]=player_x-(ball[0] + 20*raw["ball_direction"][0])
    d["ball_20step_y_diff"]=player_y-(ball[1] + 20*raw["ball_direction"][1])
    
    d["ball_dir1"]=player_x-ball[0]
    d["ball_dir2"]=player_y-ball[1]
    d["dist_ally_goal"]=((player_x+1)**2 + player_y**2)**0.5
    d["dist_opp_goal"]=((player_x-1)**2 + player_y**2)**0.5
    
    ball_dist_goal = get_dist_opp_goal(ball[0],ball[1])
    
    ally_pos_x=[]
    ally_near_opp_goal_cnt=0
    min_ind=-1
    min_dist=10**10
    for ind in range(len(raw["left_team"])):
        now_x = raw["left_team"][ind][0]
        now_y = raw["left_team"][ind][1]
        ally_pos_x.append(now_x)
        if get_dist_opp_goal(now_x,now_y) < ball_dist_goal:
            ally_near_opp_goal_cnt+=1

        if ind == player_ind:
            continue
        dist = (player_x-now_x)**2 + (player_y-now_y)**2
        if dist < min_dist:
            min_ind=ind
            min_dist=dist    
    
    ally_pos_x.sort()
    d["ally_offside_line"]=ally_pos_x[1]
    d["ally_upper_mean_x"] = np.mean(ally_pos_x[:len(ally_pos_x)//2])
    d["ally_lower_mean_x"] = np.mean(ally_pos_x[len(ally_pos_x)//2:])
    
    d["num_ally_near_opp_goal"]=ally_near_opp_goal_cnt
    d["nearest_ally_dist"]=min_dist**0.5
    d["nearest_ally_x"]=raw["left_team"][min_ind][0]
    d["nearest_ally_y"]=raw["left_team"][min_ind][1]
    d["nearest_ally_dir1"]=raw["left_team_direction"][min_ind][0]
    d["nearest_ally_dir2"]=raw["left_team_direction"][min_ind][1]
    
    opp_near_opp_goal_cnt=0
    min_ind=-1
    min_dist=10**10
    opp_pos_x = []
    for ind in range(len(raw["right_team"])):
        now_x = raw["right_team"][ind][0]
        now_y = raw["right_team"][ind][1]
        if get_dist_opp_goal(now_x,now_y) < ball_dist_goal:
            opp_near_opp_goal_cnt+=1

        opp_pos_x.append(now_x)

        dist = (player_x-now_x)**2 + (player_y-now_y)**2
        if dist < min_dist:
            min_ind=ind
            min_dist=dist
    
    d["num_opp_near_opp_goal"]=opp_near_opp_goal_cnt
    d["nearest_opp_dist_player"]=min_dist**0.5 
    d["nearest_opp_x"]=raw["right_team"][min_ind][0]
    d["nearest_opp_y"]=raw["right_team"][min_ind][1]
    d["nearest_opp_dir1"]=raw["right_team_direction"][min_ind][0]
    d["nearest_opp_dir2"]=raw["right_team_direction"][min_ind][1]
    
    min_ind=-1
    min_dist=10**10
    for ind in range(len(raw["right_team"])):
        now_x = raw["right_team"][ind][0]
        now_y = raw["right_team"][ind][1]

        dist = calc_dist(ball[0],ball[1],now_x,now_y)

        if dist < min_dist:
            min_ind=ind
            min_dist=dist  

    min_x=raw["right_team"][min_ind][0]
    min_y=raw["right_team"][min_ind][1]
    
    d["ball_nearest_opp_dist_ball"]=min_dist
    d["ball_nearest_opp_dist_player"]=calc_dist(min_x,min_y,player_x,player_y)
    d["ball_nearest_opp_x"]=min_x
    d["ball_nearest_opp_y"]=min_y
    d["ball_nearest_opp_dir1"]=raw["left_team_direction"][min_ind][0]
    d["ball_nearest_opp_dir2"]=raw["left_team_direction"][min_ind][1]

    gk_ind = raw["right_team_roles"][0]
    gk_x = raw["right_team"][gk_ind][0]
    gk_y = raw["right_team"][gk_ind][1]
        
    d["opp_keeper_x"]=gk_x
    d["opp_keeper_y"]=gk_y
    d["opp_keeper_x_player"]=player_x - gk_x
    d["opp_keeper_y_player"]=player_y - gk_y
    d["opp_keeper_dir1"]=raw["right_team_direction"][gk_ind][0]
    d["opp_keeper_dir2"]=raw["right_team_direction"][gk_ind][1]
    
    opp_pos_x.sort()
    d["offside_line"]=opp_pos_x[-2]
    d["opp_upper_mean_x"] = np.mean(opp_pos_x[:len(opp_pos_x)//2])
    d["opp_lower_mean_x"] = np.mean(opp_pos_x[len(opp_pos_x)//2:])
    d["steps_left"]=raw["steps_left"]
    d["score"]=raw["score"][0]-raw["score"][1]
    
    d["player_pressure_from_opp_x"],d["player_pressure_from_opp_y"],d["player_pressure_abs"] = pressure_to_xy(player_x,player_y,raw["right_team"])
    d["ball_pressure_from_opp_x"],d["ball_pressure_from_opp_y"],d["ball_pressure_from_opp_abs"] = pressure_to_xy(ball[0],ball[1],raw["right_team"])
    d["ball_pressure_from_ally_x"],d["ball_pressure_from_ally_y"],d["ball_pressure_from_ally_abs"] = pressure_to_xy(ball[0],ball[1],raw["left_team"])
    
    d["player_angle_dir_opp_goal"]=get_angle(player_x,player_y,1,0,d["active_dir1"],d["active_dir2"])
    d["player_angle_dir_ally_goal"]=get_angle(player_x,player_y,-1,0,d["active_dir1"],d["active_dir2"])
    
    min_ind, score,pass_x,pass_y= get_lowest_pass_score(raw["active"], raw["left_team"], raw["left_team_direction"])
    
    d["pass_x"] = pass_x
    d["pass_y"] = pass_y
    
    d["pass_score"] = score
    
    p1,p2,d["pass_pressure_from_opp_abs"]  = pressure_to_xy(pass_x,pass_y,raw["left_team"])    
    p1,p2,d["pass_pressure_from_ally_abs"] = pressure_to_xy(pass_x,pass_y,raw["right_team"])    
    
    d["pass_dist_from_player"] = calc_dist(pass_x,pass_y,player_x,player_y)
    d["pass_dist_from_ball"] = calc_dist(pass_x,pass_y,ball[0],ball[1])
    
    d["ballx_d_norm"], d["bally_d_norm"] = normalize(d["ballx_d"],d["bally_d"])
    d["active_dir1_norm"], d["active_dir2_norm"] = normalize(d["active_dir1"], d["active_dir2"])
    d["ball_10step_x_diff_norm"], d["ball_10step_y_diff_norm"] = normalize(d["ball_10step_x_diff"], d["ball_10step_y_diff"])
    d["ball_20step_x_diff_norm"], d["ball_20step_y_diff_norm"] = normalize(d["ball_20step_x_diff"], d["ball_20step_y_diff"])
    d["ball_dir1_norm"], d["ball_dir2_norm"] = normalize(d["ball_dir1"], d["ball_dir2"])
    d["nearest_ally_dir1_norm"], d["nearest_ally_dir2_norm"] = normalize(d["nearest_ally_dir1"], d["nearest_ally_dir2"])
    d["nearest_opp_dir1_norm"], d["nearest_opp_dir2_norm"] = normalize(d["nearest_opp_dir1"], d["nearest_opp_dir2"])
    d["ball_nearest_opp_dir1_norm"], d["ball_nearest_opp_dir2_norm"] = normalize(d["ball_nearest_opp_dir1"], d["ball_nearest_opp_dir2"])
    
    d["opp_keeper_x_player_norm"], d["opp_keeper_y_player_norm"] = normalize(d["opp_keeper_x_player"], d["opp_keeper_y_player"])
    d["opp_keeper_dir1_norm"], d["opp_keeper_dir2_norm"] = normalize(d["opp_keeper_dir1"], d["opp_keeper_dir2"])
    
    d["ball_d_theta"] = my_atan2(d["ballx_d_norm"], d["bally_d_norm"])
    d["active_dir_theta"] = my_atan2(d["active_dir1_norm"], d["active_dir2_norm"])
    d["ball_10step_theta"] = my_atan2(d["ball_10step_x_diff_norm"], d["ball_10step_y_diff_norm"])
    d["ball_20step_theta"] = my_atan2(d["ball_20step_x_diff_norm"], d["ball_20step_y_diff_norm"])
    d["ball_dir_theta"] = my_atan2( d["ball_dir1_norm"], d["ball_dir2_norm"])
    d["nearest_ally_dir_theta"] = my_atan2(d["nearest_ally_dir1_norm"], d["nearest_ally_dir2_norm"])
    d["nearest_opp_dir_theta"] = my_atan2(d["nearest_opp_dir1_norm"], d["nearest_opp_dir2_norm"])
    d["ball_nearest_opp_dir_theta"] = my_atan2(d["ball_nearest_opp_dir1_norm"], d["ball_nearest_opp_dir2_norm"])
    d["opp_keeper_theta"] = my_atan2(d["opp_keeper_x_player_norm"], d["opp_keeper_y_player_norm"])
    d["opp_keeper_dir_theta"] = my_atan2( d["opp_keeper_dir1_norm"], d["opp_keeper_dir2_norm"])
    
    d["sticky_sprint"] = raw["sticky_actions"][8]
    
    sub_feats=['ball_dist_xy___nearest_opp_dist_player',       'dist_ally_goal___ball_nearest_opp_dist_ball',       'dist_ally_goal___ball_nearest_opp_dist_player',       'ball_nearest_opp_dist_ball___ball_nearest_opp_dist_player',       'ballx___player_x', 'ballx___nearest_opp_x',       'ballx___opp_keeper_x', 'ballx_d___active_dir1',       'ballx_d___ball_dir1', 'ball_10step_x___player_x',       'ball_10step_x___opp_keeper_x_player',       'ball_20step_x___offside_line',       'ball_20step_x___opp_upper_mean_x',       'player_x___ally_lower_mean_x', 'player_x___nearest_opp_x',       'player_x___offside_line', 'active_dir1___ball_dir1',       'active_dir1___nearest_opp_dir1',       'active_dir1___opp_keeper_dir1',       'ball_10step_x_diff___opp_keeper_dir1',       'ball_20step_x_diff___opp_keeper_x',       'ball_dir1___nearest_opp_dir1',       'ball_dir1___ball_nearest_opp_dir1',       'ball_dir1___opp_keeper_dir1',       'ally_offside_line___ally_lower_mean_x',       'ally_upper_mean_x___offside_line',       'ally_upper_mean_x___opp_upper_mean_x',       'ally_lower_mean_x___offside_line',       'ally_lower_mean_x___opp_upper_mean_x',       'opp_keeper_x_player___opp_keeper_dir1',       'offside_line___opp_upper_mean_x', 'bally___player_y',       'bally___nearest_opp_y', 'bally_d___active_dir2',       'bally_d___ball_dir2', 'player_y___nearest_opp_y',       'active_dir2___ball_10step_y_diff',       'active_dir2___opp_keeper_dir2',       'ball_10step_y_diff___opp_keeper_dir2',       'ball_dir2___nearest_opp_dir2',       'ball_dir2___opp_keeper_dir2',       'opp_keeper_y___opp_keeper_y_player']
    for l in sub_feats:
        l1,l2 = l.split("___")
        d[l] = d[l1] - d[l2]
    
    return pd.DataFrame([d])   

def agent(obs):
    global model
    
    raw = obs['players_raw'][0]
    raw = convert_raw_y(raw)
    
    X_df = create_df(raw)
    y_pred1 = model.predict(X_df,prediction_type="Probability")
    y_pred_max = np.argmax(y_pred1, axis=1)
    
    return [int(y_pred_max[0]+1)]

In [ ]:
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle_simulations/agent/main.py", "do_nothing"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)

In [ ]:
!cd /kaggle_simulations/agent/ && tar -czvf /kaggle/working/submit.tar.gz *

In [ ]:
!rm -rf football
!rm -rf kaggle-environments